## Smart RAG
- Know what you know and what you don't know.


In [15]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

In [16]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [17]:
import warnings

warnings.filterwarnings("ignore")

In [18]:

solar_summary = """
SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective Depth Up-Scaling

We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

In [19]:
# Tools
from langchain_core.tools import tool
import requests
import os
from tavily import TavilyClient
from langchain_upstage import ChatUpstage

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])


@tool
def solar_paper_search(query: str) -> str:
    """Query for research paper about solarllm, dus, llm and general AI.
    If the query is about DUS, Upstage, AI related topics, use this.
    """
    return solar_summary


@tool
def internet_search(query: str) -> str:
    """This is for query for internet search engine like Google.
    Query for general topics.
    """
    return tavily.search(query=query)


@tool
def get_news(topic: str) -> str:
    """Get latest news about a topic.
    If users are more like recent news, use this.
    """
    # https://newsapi.org/v2/everything?q=tesla&from=2024-04-01&sortBy=publishedAt&apiKey=API_KEY
    # change this to request news from a real API
    news_url = f"https://newsapi.org/v2/everything?q={topic}&apiKey={os.environ['NEWS_API_KEY']}"
    respnse = requests.get(news_url)
    return respnse.json()


tools = [solar_paper_search, internet_search, get_news]

llm = ChatUpstage()
llm_with_tools = llm.bind_tools(tools)

In [20]:
def call_tool(tool_call):
    tool_name = tool_call["name"].lower()
    if tool_name not in globals():
        print("Tool not found", tool_name)
        return None
    selected_tool = globals()[tool_name]
    return selected_tool.invoke(tool_call["args"])

In [21]:
llm_with_tools.invoke("What is Solar LLM?").tool_calls


[{'name': 'solar_paper_search',
  'args': {'query': 'Solar LLM'},
  'id': '8d491b94-0806-4e3e-a083-ab9bd7711d16'}]

In [22]:
llm_with_tools.invoke("What is news about Seoul").tool_calls


[{'name': 'get_news',
  'args': {'topic': 'Seoul'},
  'id': '31422e2a-edf7-453f-8376-84cd7d62984a'}]

In [29]:
llm_with_tools.invoke("What's best place in Seoul?").tool_calls


[{'name': 'internet_search',
  'args': {'query': 'Best place in Seoul'},
  'id': '4c09e29e-bb34-4db2-96ba-90582421f4f3'}]

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt_template = PromptTemplate.from_template(
    """
    Please provide answer for question from the following context. 
    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [25]:
# Smart RAG, Self-Improving RAG
import os
from tavily import TavilyClient


def tool_rag(question):
    tool_calls = llm_with_tools.invoke(question).tool_calls
    print(tool_calls)
    context = ""
    for tool_call in tool_calls:
        context += str(call_tool(tool_call))

    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"context": context, "question": question})

In [26]:
tool_rag("What is Solar llm?")

[{'name': 'solar_paper_search', 'args': {'query': 'Solar llm'}, 'id': 'a00dec97-40a3-471e-8aab-28a96a5ec88b'}]


'Solar llm is a large language model (LLM) with 10.7 billion parameters, introduced as a result of depth up-scaling (DUS), which is a method for scaling LLMs that encompasses depthwise scaling and continued pretraining.'

In [27]:
tool_rag("What is news about Tesla?")

[{'name': 'get_news', 'args': {'topic': 'Tesla'}, 'id': 'ed2fca01-b6cc-49cb-a949-ee32ce0e71e5'}]


'Tesla is in the news for various reasons. Here are some of the recent news articles about Tesla:\n\n1. The company has issued a voluntary recall for nearly 4,000 Cybertrucks because the accelerator pad could get trapped, pushing the car to full speed.\n2. A federal report found that Tesla\'s Autopilot system was involved in at least 13 fatal crashes in which drivers misused the system in ways the automaker should have foreseen and done more to prevent.\n3. Tesla told investors that sales and revenue are down but that new "more affordable vehicles" will launch before mid-2025, sooner than originally planned.\n4. The company is recalling around 3,878 Cybertruck vehicles due to a faulty accelerator pedal cover that could get dislodged and trapped by the interior rim.\n5. Tesla is laying off more than 10% of its workforce, which CEO Elon Musk said was a difficult decision to reduce the company\'s global headcount.\n6. Climate protestors in Germany broke through police barricades on Friday

In [30]:
tool_rag("iPhone 13 spec?")

[{'name': 'internet_search', 'args': {'query': 'iPhone 13 spec'}, 'id': '470f191a-255e-4528-a4ed-8b3a8d6ed083'}]


'The iPhone 13 has a 6.1-inch display with a resolution of 2532 x 1170 pixels and a refresh rate of 60Hz. It is powered by the Apple A15 Bionic chipset and comes with 4GB of RAM. The iPhone 13 has a dual camera setup with 12MP sensors for both the primary and front-facing cameras. It has a 3240mAh battery and comes with 128GB of internal storage, which is not expandable. The phone is available in multiple colors including Starlight, Midnight, Blue, Pink, and Green.'

# Excercise
Solar LLM is small, so it might not give the best results for complex tasks. For those, you can use larger LLMs like GPT-4. However, for quick answers and summaries, using a small size LLM like Solar can give better performance and efficiency.
